In [2]:
import os
from nltk import word_tokenize
from nltk.corpus import stopwords
import glob
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
import io
import itertools
import networkx as nx
import nltk
import math

In [3]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
 
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
 



In [4]:
def levenshtein_distance(first, second):
    """Return the Levenshtein distance between two strings.
    Based on:
        http://rosettacode.org/wiki/Levenshtein_distance#Python
    """
    if len(first) > len(second):
        first, second = second, first
    distances = range(len(first) + 1)
    for index2, char2 in enumerate(second):
        new_distances = [index2 + 1]
        for index1, char1 in enumerate(first):
            if char1 == char2:
                new_distances.append(distances[index1])
            else:
                new_distances.append(1 + min((distances[index1],
                                             distances[index1 + 1],
                                             new_distances[-1])))
        distances = new_distances
    return distances[-1]



In [5]:
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.1, 0
    
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
       
        for ss in synsets2:
            best_score=max(score,synset.path_similarity(ss))
        # Check that the similarity could have been computed
            if best_score is not None:
                score += best_score
                count += 1
 
    # Average the values
    score /= count
    if score is Not None:
        return score
    else:
        return 0.0
        


SyntaxError: invalid syntax (<ipython-input-5-0be236b2f0a3>, line 30)

In [31]:
f= open('Article.txt', "r")

all_text = f.read()
temp=[]
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english",ignore_stopwords = True)
words = all_text.split()
words1 = []
for i in range(len(words)):
    words1.append(stemmer.stem(words[i]))
afterstem = " ".join(words1)
temp.append(afterstem)
print(temp)
str = ''.join(temp)
str=''.join(all_text)
print(str)

["almost all of us use an antivirus on our laptop or pc. and why not? with the number of attack happen everi day, like the recent wannacri ransomware, it is logic to do so. but phone and laptop are different, right? do the antivirus work on these too? let get direct to the point. the answer is no. you do not need an anti-virus applic to keep you secure. the anti-virus of your system is you. if you ask me which softwar do i use? i use my brain. that is the best line of secur in today times. android os(a major of the user are concern about android) is base on the linux kernel. it is not very easi for an applic to affect your device. there are tip you need to keep in mind and there you go- safe and secure. googl play store has a multipl line of secur check before an applic even debut on the store. so the chanc of a malici app make to the app store is neglig and by any means, if the applic does reach the app store, googl keep a track and will take down the app as soon as possible. even if 

In [32]:
def build_graph(nodes):
    """Return a networkx graph instance.
    :param nodes: List of hashables that represent the nodes of a graph.
    """
    gr = nx.Graph()  # initialize an undirected graph
    gr.add_nodes_from(nodes)
    nodePairs = list(itertools.combinations(nodes, 2))

    # add edges to the graph (weighted by Levenshtein distance)
    for pair in nodePairs:
        firstString = pair[0]
        secondString = pair[1]
        levDistance = levenshtein_distance(firstString, secondString)
        
        WNScore=sen(firstString, secondString)
        
        Distance=(levDistance+WNScore)/2.0
        gr.add_edge(firstString, secondString, weight=Distance)

    return gr

In [33]:
from nltk.corpus import wordnet
def sen(first,second):
    score, count = 0.0,0
    for word in first:
        for each in second:
            x=wordnet.synsets(word)[0]
            y=wordnet.synsets(each)[0]
            if(x and y):
                score+=x.wup_similarity(y)
                count+=1
                return score/count
            else:
                return 0

In [107]:
def extract_sentences(text,l):
    
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sentence_tokens = sent_detector.tokenize(text.strip())
    graph = build_graph(sentence_tokens)
    calculated_page_rank = nx.pagerank(graph, weight='weight')
    
    # most important sentences in ascending order of importance
    sentences = sorted(calculated_page_rank, key=calculated_page_rank.get,
                       reverse=True)
    l=int(l)
    # return a 100 word summary
    summary = ' '.join(sentences)
    summary_words = summary.split()
    
    summary_words = summary_words[0:l]
    
    summary = ' '.join(summary_words)

    return summary


In [121]:
from nltk import tokenize
def extract_sentences1(text,l):
    
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sentence_tokens = sent_detector.tokenize(text.strip())
    graph = build_graph(sentence_tokens)
    calculated_page_rank = nx.pagerank(graph, weight='weight')
    
    # most important sentences in ascending order of importance
    sentences = sorted(calculated_page_rank, key=calculated_page_rank.get,
                       reverse=True)
    l=int(l)
    # return a 100 word summary
    summary = ' '.join(sentences)
    
    x=tokenize.sent_tokenize(summary)
    x=x[0:3]
    x=x[:-1]
    print(x)
   

    summary = ' '.join(x)

    return summary


In [122]:
import re
count=len(re.findall(r'\w+', str))
count=int(count/3)
for each in range(len(str)-count):
    if str[count+each] is '.':
        break

summary = extract_sentences1(str,count)
print(summary)    

['If you need to download an application that does not exist in the Play Store, try to go for the reputed App stores like Amazon App Store or APKMirror, otherwise, avoid downloading it and look for better alternatives.', 'So the chance of a malicious app making to the app store is negligible and by any means, if the application does reach the app store, Google keeps a track and will take down the app as soon as possible.']
If you need to download an application that does not exist in the Play Store, try to go for the reputed App stores like Amazon App Store or APKMirror, otherwise, avoid downloading it and look for better alternatives. So the chance of a malicious app making to the app store is negligible and by any means, if the application does reach the app store, Google keeps a track and will take down the app as soon as possible.


In [102]:
import re
line = " I am having a very nice day helo hoealek keawo;ea."
count = len(re.findall(r'\w+', line))
print (count)

11
